In [1]:
import sys
sys.path.insert(0,'/exp/rhuang/meta/icefall/egs/librispeech/ASR/conformer_ctc2_noseg')
sys.path.insert(0,'/exp/rhuang/meta/k2/k2/python')
sys.path.insert(0,'/exp/rhuang/meta/k2/temp.linux-x86_64-cpython-310/lib')
sys.path.insert(0,'/exp/rhuang/meta/icefall')

In [2]:
%cd /exp/rhuang/meta/icefall/egs/librispeech/ASR
%pwd  #look at the current work dir

/exp/rhuang/meta/icefall/egs/librispeech/ASR


'/exp/rhuang/meta/icefall/egs/librispeech/ASR'

In [4]:
from train import *

In [13]:
# main()

parser = get_parser()
LibriSpeechAsrDataModule.add_arguments(parser)
args = parser.parse_args(args=[])

args.exp_dir = "/home/rhuang25/work/icefall/egs/librispeech/ASR/zipformer_no_seg/exp-test/"
args.bpe_model = "/home/rhuang25/work/icefall/egs/librispeech/ASR/data/lang_bpe_500/bpe.model"
args.epoch = 2
args.avg = 1

# args.exp_dir = "/home/rhuang25/work/icefall/egs/librispeech/ASR/zipformer/exp-ctc-rnnt"
# args.bpe_model = "/home/rhuang25/work/icefall/egs/librispeech/ASR/data/lang_bpe_500/bpe.model"
# args.epoch = 40
# args.avg = 1

args.manifest_dir = "/home/rhuang25/work/icefall/egs/librispeech/ASR/data/fbank/"
args.max_duration = 400
args.world_size = 1
args.start_epoch = 2
args.full_libri = False
args.use_transducer = False
args.use_ctc = True
args.ctc_loss_scale = 1.0
args.ctc_beam_size = 4

args.exp_dir = Path(args.exp_dir)
args.manifest_dir = Path(args.manifest_dir)

world_size = args.world_size
assert world_size >= 1

rank=0
world_size=1
args=args

In [14]:
# run

params = get_params()
params.update(vars(args))

fix_random_seed(params.seed)
if world_size > 1:
    setup_dist(rank, world_size, params.master_port)

# setup_logger(f"{params.exp_dir}/log/log-train")
# logging.info("Training started")

# if args.tensorboard and rank == 0:
#     tb_writer = SummaryWriter(log_dir=f"{params.exp_dir}/tensorboard")
# else:
#     tb_writer = None

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda", rank)
logging.info(f"Device: {device}")

sp = spm.SentencePieceProcessor()
sp.load(params.bpe_model)

# <blk> is defined in local/train_bpe_model.py
params.blank_id = sp.piece_to_id("<blk>")
params.vocab_size = sp.get_piece_size()

if not params.use_transducer:
    params.ctc_loss_scale = 1.0

logging.info(params)

logging.info("About to create model")
model = get_model(params)

num_param = sum([p.numel() for p in model.parameters()])
logging.info(f"Number of model parameters: {num_param}")

assert params.save_every_n >= params.average_period
model_avg: Optional[nn.Module] = None
if rank == 0:
    # model_avg is only used with rank 0
    model_avg = copy.deepcopy(model).to(torch.float64)

assert params.start_epoch > 0, params.start_epoch
checkpoints = load_checkpoint_if_available(
    params=params, model=model, model_avg=model_avg
)

model.to(device)
if world_size > 1:
    logging.info("Using DDP")
    model = DDP(model, device_ids=[rank], find_unused_parameters=True)

optimizer = ScaledAdam(
    get_parameter_groups_with_lrs(model, lr=params.base_lr, include_names=True),
    lr=params.base_lr,  # should have no effect
    clipping_scale=2.0,
)

scheduler = Eden(optimizer, params.lr_batches, params.lr_epochs)

if checkpoints and "optimizer" in checkpoints:
    logging.info("Loading optimizer state dict")
    optimizer.load_state_dict(checkpoints["optimizer"])

if (
    checkpoints
    and "scheduler" in checkpoints
    and checkpoints["scheduler"] is not None
):
    logging.info("Loading scheduler state dict")
    scheduler.load_state_dict(checkpoints["scheduler"])

if params.print_diagnostics:
    opts = diagnostics.TensorDiagnosticOptions(
        512
    )  # allow 4 megabytes per sub-module
    diagnostic = diagnostics.attach_diagnostics(model, opts)

if params.inf_check:
    register_inf_check_hooks(model)

librispeech = LibriSpeechAsrDataModule(args)

train_cuts = librispeech.train_clean_100_cuts()
if params.full_libri:
    train_cuts += librispeech.train_clean_360_cuts()
    train_cuts += librispeech.train_other_500_cuts()

# train_cuts = librispeech.test_clean_cuts() + librispeech.test_other_cuts()

def remove_short_and_long_utt(c: Cut):
    # Keep only utterances with duration between 1 second and 20 seconds
    #
    # Caution: There is a reason to select 20.0 here. Please see
    # ../local/display_manifest_statistics.py
    #
    # You should use ../local/display_manifest_statistics.py to get
    # an utterance duration distribution for your dataset to select
    # the threshold
    if c.duration < 1.0 or c.duration > 20.0:
        # logging.warning(
        #     f"Exclude cut with ID {c.id} from training. Duration: {c.duration}"
        # )
        return False

    # In pruned RNN-T, we require that T >= S
    # where T is the number of feature frames after subsampling
    # and S is the number of tokens in the utterance

    # In ./zipformer.py, the conv module uses the following expression
    # for subsampling
    T = ((c.num_frames - 7) // 2 + 1) // 2
    tokens = sp.encode(c.supervisions[0].text, out_type=str)

    if T < len(tokens):
        logging.warning(
            f"Exclude cut with ID {c.id} from training. "
            f"Number of frames (before subsampling): {c.num_frames}. "
            f"Number of frames (after subsampling): {T}. "
            f"Text: {c.supervisions[0].text}. "
            f"Tokens: {tokens}. "
            f"Number of tokens: {len(tokens)}"
        )
        return False

    return True

train_cuts = train_cuts.filter(remove_short_and_long_utt)

# get long text for each recording
libri_long_text = get_long_text(train_cuts, sp=sp, make_fst=True)
logging.info(f"len(libri_long_text) = {len(libri_long_text)}")
my_args = {"libri_long_text": libri_long_text}

get_model_scrach_space(model, k="subsampling_factor", v=params.subsampling_factor, set_value=True)
get_model_scrach_space(model, k="ctc_beam_size", v=params.ctc_beam_size, set_value=True)
get_model_scrach_space(model, k="sp", v=sp, set_value=True)
get_model_scrach_space(model, k="params", v=params, set_value=True)

if params.start_batch > 0 and checkpoints and "sampler" in checkpoints:
    # We only load the sampler's state dict when it loads a checkpoint
    # saved in the middle of an epoch
    sampler_state_dict = checkpoints["sampler"]
else:
    sampler_state_dict = None

train_dl = librispeech.train_dataloaders(
    train_cuts, sampler_state_dict=sampler_state_dict
)

valid_cuts = librispeech.dev_clean_cuts()
valid_cuts += librispeech.dev_other_cuts()
valid_dl = librispeech.valid_dataloaders(valid_cuts)

# if not params.print_diagnostics:
#     scan_pessimistic_batches_for_oom(
#         model=model,
#         train_dl=train_dl,
#         optimizer=optimizer,
#         sp=sp,
#         params=params,
#     )

scaler = GradScaler(enabled=params.use_fp16, init_scale=1.0)
if checkpoints and "grad_scaler" in checkpoints:
    logging.info("Loading grad scaler state dict")
    scaler.load_state_dict(checkpoints["grad_scaler"])

100%|██████████| 585/585 [00:20<00:00, 28.04it/s]


In [15]:
epoch = params.start_epoch

scheduler.step_epoch(epoch - 1)
fix_random_seed(params.seed + epoch - 1)
train_dl.sampler.set_epoch(epoch - 1)

# if tb_writer is not None:
#     tb_writer.add_scalar("train/epoch", epoch, params.batch_idx_train)

params.cur_epoch = epoch

In [16]:
params=params
model=model
model_avg=model_avg
optimizer=optimizer
scheduler=scheduler
sp=sp
train_dl=train_dl
valid_dl=valid_dl
scaler=scaler
tb_writer=None
world_size=world_size
rank=rank
my_args=my_args

In [17]:
# train_one_epoch
model.train()

tot_loss = MetricsTracker()

saved_bad_model = False

def save_bad_model(suffix: str = ""):
    save_checkpoint_impl(
        filename=params.exp_dir / f"bad-model{suffix}-{rank}.pt",
        model=model,
        model_avg=model_avg,
        params=params,
        optimizer=optimizer,
        scheduler=scheduler,
        sampler=train_dl.sampler,
        scaler=scaler,
        rank=0,
    )

get_model_scrach_space(model, k="log_priors", v=None, set_value=True)
get_model_scrach_space(model, k="priors_T", v=0, set_value=True)
get_model_scrach_space(model, k="rank", v=rank, set_value=True)

batch_idx, batch = next(enumerate(train_dl))

params.batch_idx_train += 1
batch_size = len(batch["supervisions"]["text"])

supervisions = batch["supervisions"]        
my_args["supervisions"] = supervisions

params=params
model=model
sp=sp
batch=batch
is_training=True
my_args=my_args

In [18]:
# compute_loss

device = model.device if isinstance(model, DDP) else next(model.parameters()).device
feature = batch["inputs"]
# at entry, feature is (N, T, C)
assert feature.ndim == 3
feature = feature.to(device)

supervisions = batch["supervisions"]
feature_lens = supervisions["num_frames"].to(device)

batch_idx_train = params.batch_idx_train
warm_step = params.warm_step

if my_args is None:
    texts = batch["supervisions"]["text"]
    y = sp.encode(texts, out_type=int)
    y = k2.RaggedTensor(y)
else:
    libri_long_text = my_args["libri_long_text"]
    cuts = batch['supervisions']['cut']
    y = [libri_long_text[tuple(get_uid_key(c.id)[:2])] for c in cuts]
    y = (k2.RaggedTensor([[0]] * feature.size(0)), y)
    # k2.ragged.create_ragged_tensor([ [1, 2], [5], [], [9] ])
    # k2.ragged.create_ragged_tensor([ [1, 2], [5], [], [9] ]).tolist()
    get_model_scrach_space(model, k="cuts", v=cuts, set_value=True)

get_model_scrach_space(model, k="texts", v=batch["supervisions"]["text"], set_value=True)

x=feature
x_lens=feature_lens
y=y
prune_range=params.prune_range
am_scale=params.am_scale
lm_scale=params.lm_scale
my_args=my_args

In [19]:
if my_args is not None:
    y, y_list = y

# self.scratch_space["my_args"] = my_args

assert x.ndim == 3, x.shape
assert x_lens.ndim == 1, x_lens.shape
assert y.num_axes == 2, y.num_axes

assert x.size(0) == x_lens.size(0) == y.dim0, (x.shape, x_lens.shape, y.dim0)

# model.eval()
model.train()

# Compute encoder outputs
encoder_out, encoder_out_lens = model.forward_encoder(x, x_lens)

row_splits = y.shape.row_splits(1)
y_lens = row_splits[1:] - row_splits[:-1]

targets = y.values   # on CPU

encoder_out=encoder_out
encoder_out_lens=encoder_out_lens
targets=y_list
target_lengths=y_lens

In [20]:
# forward_ctc_long_form

from icefall.decode import get_lattice, one_best_decoding
from icefall.utils import get_alignments, get_texts


ctc_output = model.ctc_output(encoder_out)  # (N, T, C)

supervision_segments, texts, indices = model.encode_supervisions(targets, target_lengths, encoder_out_lens)

y_list = targets
_y_list = [y_list[i] for i in indices.tolist()]
decoding_graph = k2.create_fsa_vec(_y_list)
decoding_graph = k2.arc_sort(decoding_graph)
decoding_graph = decoding_graph.to(encoder_out.device)

log_probs = ctc_output

lattice = get_lattice(
    nnet_output=log_probs,
    decoding_graph=decoding_graph,
    supervision_segments=supervision_segments,
    search_beam=15,
    output_beam=6,
    min_active_states=30,
    max_active_states=10000,
    subsampling_factor=model.scratch_space["subsampling_factor"],
)

for i in range(10):
    model.check_lattice2(lattice, indices, i)

[ref] I NEVER SELL GOODS WITHOUT KNOWING WHERE I CAN FIND THEM WHEN I WANT THEM AND IF THESE FELLOWS TRY TO PUT THEIR FOREFEET IN THE TROUGH OR START ANY SHOVING AND CROWDING THEY'RE GOING TO FIND ME FORGETTING MY TABLE MANNERS TOO
[hyp] I NEVER SELL GOODS WITHOUT KNOWING WHERE I CAN FIND THEM WHEN I WANT THEM AND IF THESE FELLOWS TRY TO PUT THEIR FOREFEET IN THE TROUGH OR START ANY SHOVING AND CROWDING THEY'RE GOING TO FIND ME FORGETTING MY TABLE MANNERS TOO
[cut] 3242-67168-0000-6032
[ref] NOTHING TO LAY TO HER CHARGE BUT HER BEING THE INVOLUNTARY UNCONSCIOUS OBJECT OF A DECEPTION WHICH HIS PRIDE COULD NOT PARDON AND WHICH A BETTER PRIDE WOULD HAVE BEEN ASHAMED TO OWN
[hyp] NOTHING TO LAY TO HER CHARGE BUT HER BEING THE INVOLUNTARY UNCONSCIOUS OBJECT OF A DECEPTION WHICH HIS PRIDE COULD NOT PARDON AND WHICH A BETTER PRIDE WOULD HAVE BEEN ASHAMED TO
[cut] 19-227-0035-7874_sp1.1
[ref] I WAS AFRAID TO APPROACH OH IS MAMMA TAKEN AWAY WHERE IS SHE WHERE HAVE THEY BROUGHT HER TO I WAS UTTE

In [21]:
print(params)

{'best_train_loss': 0.13572675826983707, 'best_valid_loss': 0.2689881270166117, 'best_train_epoch': 14, 'best_valid_epoch': 14, 'batch_idx_train': 10603, 'log_interval': 50, 'reset_interval': 200, 'valid_interval': 1000, 'feature_dim': 80, 'subsampling_factor': 4, 'warm_step': 2000, 'env_info': {'k2-version': '1.24.4', 'k2-build-type': 'Debug', 'k2-with-cuda': True, 'k2-git-sha1': '415fe1f446fffe1d9e7219b5033966294c0b430c', 'k2-git-date': 'Wed Dec 20 21:38:57 2023', 'lhotse-version': '1.20.0.dev+git.c6788492.clean', 'torch-version': '2.1.2', 'torch-cuda-available': True, 'torch-cuda-version': '11.8', 'python-version': '3.1', 'icefall-git-branch': 'noseg', 'icefall-git-sha1': '9c12663-dirty', 'icefall-git-date': 'Sun Feb 4 04:05:36 2024', 'icefall-path': '/scratch4/skhudan1/rhuang25/icefall', 'k2-path': '/scratch4/skhudan1/rhuang25/k2/k2/python/k2/__init__.py', 'lhotse-path': '/scratch4/skhudan1/rhuang25/lhotse/lhotse/__init__.py', 'hostname': 'gpu08', 'IP address': '172.20.30.8'}, 'wor

In [3]:
from train import *

/home/hltcoe/rhuang/mambaforge/envs/aligner5/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
